In [1]:
from __future__ import print_function
import ipywidgets as widgets 
import matplotlib.pyplot as plt
import numpy as np

# Exclusively Gaming the system

**Assumptions**

* JC gets no benefit from the job
* The job has probability p1 of resulting in a1 and 1-p1 of resulting in a2
* Since JC knows both results it verifies every result at essentially 0 cost, pays RP for a1 and submits a2 for mediation. 
* As worst case scenario for JC that RP never fails, RP is only blamed by mediator messing up.

In [2]:
# Mediator Parameters
wn = widgets.IntSlider(min=0,max=10,step=1,value=4)
wPR = widgets.FloatSlider(min=0,max=300,step=.5,value=1) #Penalty Rate
wNDPR = widgets.FloatSlider(min=0,max=300,step=.5,value=1)#Non-deterministic Penalty rate

def MNDU():
    available = JCmediatorPrice + RPmediatorPrice
    mediationFee = p*(p2+(1-q))*(p1**M_n)*(RPdeposit-price)
    ND_Fee = p*(p2+(1-q))*(1-(p1**M_n))*(NDdeposit-price) 
    return available + mediationFee + ND_Fee




In [3]:
#Solver
def SNDU(): #solver nondeterminisitc utility
    totMatchingFee = RPMatchingFee + JCMatchingFee
    return totmatchingFee

In [4]:
# JC parameters
wPrice = widgets.IntSlider(min=0,max=10000,step=1,value=1)
JCgasPrice = 0
wp = widgets.FloatSlider(min=0,max=1,step=.1,value=1) # how often JC verifies
wcj= widgets.FloatSlider(min=0,max=1,step=.1,value=0) # cost for JC to verify
wBenefit=widgets.IntSlider(min=0,max=10000,step=1,value=1)
matchPrice = 1
mediatorPrice = 1

def JCNDU(M_n,M_PR,M_NDPR, p1,price,p=1,cj=0, q=1):
    p2 = 1-p1
    deposit = price * M_PR * M_n
    NDdeposit = price * M_NDPR
    damages = price + JCgasPrice
    
    verifyCost = -p*cj # chance to check job (p) and recompute (cj)
    A1payout = -q*p1*price # RP correctly (q) returns answer 1 (p1) and pay up to max_price (-price). LATER ADD BENEFIT
    compensation = p*(p2+(1-q))*(p1**M_n)*damages
    fine = - p*(p2+(1-q))*(1-(p1**M_n))*NDdeposit #RP returns answer 2 correctly or responds incorrectly (p2+(1-q)) and send to mediator who finds out nonDeterminism (1-p1^n), pay fine and reward (f2+r)
    
    JCnonDet = verifyCost + A1payout + compensation + fine
    return JCnonDet

In [5]:
# RP parameters
RPgasPrice = 0
wq = widgets.FloatSlider(min=0,max=1,step=.1,value=1) # probability that RP runs correctly
wroi = widgets.FloatSlider(min=0,max=1,step=.1,value=.5) # what percent of reward is profit
winsurance = widgets.FloatSlider(min=0,max=1,step=.1,value=0) # % of deposit that can be recovered because execution is so cheap.
matchPrice = 1
mediatorPrice = 1

def RPNDU(M_n,M_PR,M_NDPR, p1,price,p=1, q=1, roi=0, Insurance=0):
    p2 = 1-p1
    deposit = price * M_PR * M_n
    damages = price + RPgasPrice
    execCost = - price*(1-roi) + deposit*Insurance

    noVerifypayout = (1-p)*price #compute answer 1 (p1) correctly (q) receive reward (price)
    
    P1payout = p*p1*q*price #compute answer 1 (p1) correctly (q) receive reward (price)
    
    P2payout = p*p2*q*(1-p1**M_n)*damages #compute answer 2 (p2) correctly (q) get sent to mediation, who finds nondeterminism (1-p1^n) and recieve reward (r) and nonDet bonus (f2-f)*k kickback=k

    notP1cost = - p*(p2+1-q)*(p1**M_n)*deposit # compute answer 2 correctly or compute incorrectly (p2+1-q)  get sent to mediation, who doesn't find nondeterminsm (p1^n) and get fined (f)
    
    failComp =   p*(1-q)*(1-p1**M_n)*damages# compute incorrectly (1-q) get sent to mediation, finds nondeterminism (1-p1^n) and bad RP? Pay RP for ND? Not sure its bad it could be totally random.
    
    RCnonDet = execCost + noVerifypayout + P1payout + P2payout + notP1cost + failComp
    return RCnonDet

#Resource Provider nondeterministic utility strategy D (decieving?), basically cheating
def RPNDUD(M_n,M_PR,M_NDPR, Cj, p1,price,p=1, q=1, roi=0, Insurance=0):
    p2 = 1-p1
    deposit = price * M_PR * M_n
    damages = price + RPgasPrice
    #execCost = - price*(1-roi) + deposit*Insurance

    #Cj is cost of execution for fraudulant job, picking a random number or "close" value
    execCost = - Cj + deposit*Insurance
    
    noVerifypayout = (1-p)*price #compute answer 1 (p1) correctly (q) receive reward (price)
    
    P1payout = p*p1*q*price #compute answer 1 (p1) correctly (q) receive reward (price)
    
    P2payout = p*p2*q*(1-p1**M_n)*damages #compute answer 2 (p2) correctly (q) get sent to mediation, who finds nondeterminism (1-p1^n) and recieve reward (r) and nonDet bonus (f2-f)*k kickback=k

    notP1cost = - p*(p2+1-q)*(p1**M_n)*deposit # compute answer 2 correctly or compute incorrectly (p2+1-q)  get sent to mediation, who doesn't find nondeterminsm (p1^n) and get fined (f)
    
    failComp =   p*(1-q)*(1-p1**M_n)*damages# compute incorrectly (1-q) get sent to mediation, finds nondeterminism (1-p1^n) and bad RP? Pay RP for ND? Not sure its bad it could be totally random.
    
    RCnonDet = execCost + noVerifypayout + P1payout + P2payout + notP1cost + failComp
    return RCnonDet

In [6]:
def plot(M_n,M_PR,M_NDPR,price,p=1,cj=0, q=1,roi=1,Insurance=0):   
    plt.figure(2,figsize=(10,5))
    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    x,step = np.linspace(0,1,51,retstep=True)
    print("step size: %s" %step)
    yJND = list(map(lambda p1: JCNDU(M_n,M_PR,M_NDPR, p1,price,p=p,cj=cj, q=q),x))
    yRND = list(map(lambda p1: RPNDU(M_n,M_PR,M_NDPR, p1,price,p=p, q=q,roi=roi,Insurance=Insurance),x))
    plt.plot(x,yJND, label="JUND")
    plt.plot(x,yRND, label="RUND")
    plt.xlabel('P(a1)')
    plt.ylabel('Util') 
    
    plt.legend()
    plt.show
    
    deposit = price * M_PR * M_n
    execCost = price*(1-roi)
    print("execCost: %s" %execCost)
    requstedPrice = execCost + deposit*Insurance   
    print("requstedPrice: %s" %requstedPrice)
    print("price > requstedPrice, Matchable? : %s" %(price>requstedPrice))
    
    print("max ND JC utility: %s" %max(yJND))
    print("P(a1) for JCmax/RPmin util: %s" %x[yJND.index(max(yJND))])
    print("min RP utility: %s" %min(yRND))
    
    print("RP is losing: %s" %(min(yRND)<0))

In [7]:
interactive_plot = widgets.interactive(plot, M_n=wn,M_PR=wPR,M_NDPR=wNDPR,price=wPrice,p=wp,cj=wcj, q=wq,roi=wroi,Insurance=winsurance);
output = interactive_plot.children[-1]
output.layout.height = '600px'
interactive_plot

interactive(children=(IntSlider(value=4, description='M_n', max=10), FloatSlider(value=1.0, description='M_PR'…

# Benefit Analysis

This section analyzes when the JC is sending real jobs that provide it value when they are successfully completed. The JC is attempting to increase its utility by adding a non-deterministic piece to the jobs in an effort to have some jobs completed for free by accusing the RP of failing to execute them correctly. This could be done by changing the input so that some parts of the code are normally dorment and only activated when a certain data input is provided.

### Protocol Following model
If the JC follows the protocol its utility is the benefit, or value, it receives from the Job execution minus the amount it pays to the RP. This is assuming that the RP is playing by the rules of the game, otherwise the JC could have better utility since **price** will be refunded and **benefit** will still be recieved since the mediator will return the result. 

This model is further simplified by the assumption that a Job can be checked for approximately free, and that the JC can check the result of every job. This is reasonable since such jobs exist and we are interested in identifying the worst case scenario we need to protect against. 

In [8]:
def protocolU(benefit, price):
    return benefit - price

### Non-Deterministic Model
The key element of this model is the amount of utility the JC can get for **free** from the system. 
The free utility is the probability that the RP got result 2 multiplied by the probability that the mediator got result 1 $n$ times multiplied by the job benefit, minus the probability that the RP got result 2 multiplied by the probability that the mediator did not get result 1 for any of $n$ executions multiplied by the deposit, whose value is determined by the price set by the JC multiplied by the non-deterministic penalty rate set by the Mediator. 

In [9]:
def NDU(M_n,M_NDPR, p1,price,benefit):
    p2 = 1-p1
    NDdeposit = price * M_NDPR
    paid = p1*(benefit-price)
    free = p2*((p1**M_n)*benefit - (1-p1**M_n)*NDdeposit)
    total = paid + free
    return total
    

### Evaluation

In [10]:
def plot2(M_n,M_NDPR,  price,benefit):
    x,step = np.linspace(0,1,51,retstep=True)
    yPU = list(map(lambda p1: protocolU(benefit,price),x))
    yNDU = list(map(lambda p1: NDU(M_n,M_NDPR, p1,price,benefit),x))
    
    
    plt.figure(3,figsize=(10,5))    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    plt.plot(x,yPU, label="PU")
    plt.plot(x,yNDU, label="NDU")
    plt.xlabel('P(a1)')
    plt.ylabel('Util') 
    
    plt.legend()
    plt.show
    
    print("Protocol util: %s" %yPU[0])
    print("max ND util: %s" %max(yNDU))
    print("ND improvement over Protocol: %s" %(max(yNDU)-yPU[0]))    
    print("%% improvement : %s %%" %(100*abs((max(yNDU)-yPU[0])/yPU[0])))
    print("optimal P(p1): %s" %x[yNDU.index(max(yNDU))])
    
    

In [11]:
interactive_plot2 = widgets.interactive(plot2, M_n=wn,M_NDPR=wNDPR, price=wPrice,benefit=wBenefit);
output = interactive_plot2.children[-1]
output.layout.height = '600px'
interactive_plot2

#M_n = 4
#NDPR = 12

interactive(children=(IntSlider(value=4, description='M_n', max=10), FloatSlider(value=1.0, description='M_NDP…

### Bounding the problem
$free = (1-P)P^nb - (1-P)(1-P^n)Rc$

**Derivative**

$\frac{d}{dP}(free) = \frac{d}{dP} (1-P)P^nb - (1-P)(1-P^n)Rc$

$0 = \frac{d}{dP} (P^nb - P^{n+1} - Rc + P^nRc + PRc - P^{n+1}Rc$)

$0 = nP^{n-1}b - (n+1)P^{n}b - 0 + nP^{n-1}Rc + Rc - (n+1)P^{n}Rc$

**Collect Terms**

$ nP^{n-1}b + nP^{n-1}Rc + Rc =  (n+1)P^{n}b + (n+1)P^{n}Rc$

$ nP^{n-1}(b+Rc) + Rc =  (n+1)P^{n}(b+Rc)$

$ nP^{n-1} + \frac{Rc}{b+Rc} =  (n+1)P^{n}$


$\frac{Rc}{b+Rc} + nP^{n-1} - (n+1)P^n = 0 $

**Let penalty rate R=0**

benefit and cost become irrelevant and we are left with:

$nP^{n-1}  = (n+1)P^n $

**Simplify**

$\frac{n}{n+1} = P$

So the maximum the JC can make with no penalty rate will be when $P = \frac{n}{n+1}$

$P$ is the liklihood of getting answer 1 and $n$ is the number of times the mediator verifies, so we can force the JC to cheat less and limit its additional utility. 

For example if $n=4$ then $P=.8$, so for optimal gains JC will make it so that a1 appears 80% of the time, anything else would give it less reward, so if we assume the worst case and handle it, anything less severe will be handled as well. 

**If R is not 0**

any change to $b$ or $c$ will never make lower probability of a1 more optimal. 

**For $b<c$ and R=1**

The normal protocol loses, but the ND protocol loses somewhat less until $b=.9646c$ approximately, as which point ND can win slightly. 

It can win .53 utility units when $b=97$ and $c=100$. If these numbers are scaled by factors of 10 the utility decreases. 

**For $b=c$ and R=1**

Optimal is when probability of a1 is 92%. 
Normal protocal loses, and ND protocol wins 3.46% of benfit/price as utility. 

**For $b>c$ and R=1**

As $b$ grows the advantage over the protocol decreases compared to $b=c$

When $b=2.75c$ the added benefit is less than 1%. 

**Things get worse for JC if R increases.**
The worst case for the system is $b=c$ as the JC can get the most advantage. This can be mitigated by assuming JC is cheating and taking a 3.46% tax on the price it includes, or increasing R to, apparently, 12, after which point ND can't win and is only better than protocol if $b=.88c$ or less (meaning its losing badly since it needed 96% to win in the first place).


